In [1]:
import requests
import csv

In [2]:
import pandas as pd

In [21]:
from bs4 import BeautifulSoup as BeautifulSoup

In [30]:
local_html_file_path = '/home/fiend/Srcode/betpro/data/history_332640219.html'

In [ ]:
with open(local_html_file_path, 'r', encoding='utf-8') as file:
    html_content = file.read()
# Create BeautifulSoup object
soup = BeautifulSoup(html_content, 'html.parser')

# Open a CSV file in write mode
with open('output.csv', 'w', newline='', encoding='utf-8') as csv_file:
    # Create a CSV writer
    csv_writer = csv.writer(csv_file)

    # Write header row
    csv_writer.writerow(['Bet Slip', 'Bet Type', 'Game', 'Odds', 'Status'])

    # Find all div elements with class 'cupHisNew'
    cup_his_new_divs = soup.find_all('div', class_='cupHisNew')

    # Iterate through each 'cupHisNew' div
    for index, cup_his_new_div in enumerate(cup_his_new_divs, start=1):
        # Extract relevant information from the div
        bet_slip_number = cup_his_new_div.find('b').text.strip()
        time = cup_his_new_div.find('time').text.strip()
        bet_label = cup_his_new_div.find('label', class_='hisName').text.strip()
        odds = cup_his_new_div.find('div', class_='hisCof').text.strip()

        # Extract table data
        table_rows = cup_his_new_div.find('table', class_='table_prop').find_all('tr')[2:]
        for row in table_rows:
            # Check if there are enough 'td' elements before trying to access them
            if len(row.find_all('td')) >= 5:
                game = row.find_all('td')[1].text.strip()
                bet_type = row.find_all('td')[2].text.strip()
                odds = row.find_all('td')[3].text.strip()
                status = row.find_all('td')[4].text.strip()

                # Write row data to CSV
                csv_writer.writerow([bet_slip_number, bet_type, game, odds, status])
            else:
                print(f"Bet Slip {index} - Incomplete row data")

In [89]:
#group by Main Sport and Status
df_grouped = df.groupby(['Main Sport', 'Status']).size().reset_index(name='Count')

In [3]:
# Split the 'Game' column into separate columns
df[['Sport', 'Event Details', 'Date Time']] = df['Game'].str.split('\n+', expand=True)

# Drop the original 'Game' column if no longer needed
df.drop('Game', axis=1, inplace=True)

# Strip leading and trailing whitespaces from the new columns
#df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)

# Split 'Date Time' column into 'Date' and 'Time'
df[['Date', 'Time']] = df['Date Time'].str.split('|', expand=True)
df.drop('Date Time', axis=1, inplace=True)

# Split 'Sport' column into 'Main Sport' and 'Sub Sport'
df[['Main Sport', 'Sub Sport']] = df['Sport'].str.split('. ', n=1, expand=True)
df.drop('Sport', axis=1, inplace=True)

# Split 'Event Details' column into 'Home' and 'Away'
df[['Home', 'Away']] = df['Event Details'].str.split('-', n=1, expand=True)
df.drop('Event Details', axis=1, inplace=True)

In [5]:
df.drop('Bet Slip', axis=1, inplace=True)

In [ ]:
df.set_index('Date', inplace=True)
df.head(10)

In [ ]:
#reoder columns to 'Time' 'Bet Type',  'Main Sport', 'Sub Sport', 'Home', 'Away' 'Odds', 'Status'
df = df[['Time', 'Bet Type', 'Main Sport', 'Sub Sport', 'Home', 'Away', 'Odds', 'Status']]
df.head(10)

date : "02.01.2024", Time "06:30", Pick: "Total Over (224.5)", Sport Type: "Basketball", Match: "NBA	Los Angeles Clippers vs Miami Heat (Including Overtime)", Odd: "1.95", Outcome: "win"

In [16]:
#save the data to csv
df.to_csv('1xbet_history.csv')

In [3]:
df = pd.read_csv('1xbet_history.csv', parse_dates=['Date'], dayfirst=True)

### most profitable sport

In [ ]:
# Get the number of wins per sport type
total_counts = df.groupby('Main Sport').size()

# Filter the DataFrame for wins and count per sport type
win_counts = df[df['Status'] == 'win'].groupby('Main Sport').size()

# Create a new DataFrame with win counts and total counts
win_stats = pd.DataFrame({
    'Win Count': win_counts,
    'Total Count': total_counts
})

# Calculate win percentage and add it as a new column in the DataFrame
win_stats['Win Percentage'] = (win_stats['Win Count'] / win_stats['Total Count']) * 100
win_stats.sort_values('Total Count', ascending=False)

In [4]:
win_stats.sort_values('Total Count', ascending=False)

,Win Count,Total Count,Win Percentage
Main Sport,,,
Football,474.0,916,51.746725
Basketball,115.0,237,48.523207
Volleyball,14.0,21,66.666667
Tennis,7.0,12,58.333333
Ic,3.0,9,33.333333
Tabl,2.0,3,66.666667
Baseball,2.0,2,100.000000
America,NaN,1,NaN
Cricket,1.0,1,100.000000


### Average winning odds per sport

In [7]:
# If 'Odd' is not numeric, you should convert it first using pd.to_numeric
df['Odd'] = pd.to_numeric(df['Odds'], errors='coerce')

# Filter the DataFrame for wins
wins_df = df[df['Status'] == 'win']

# Group by 'Sport Type' and calculate the mean of 'Odd'
average_winning_odds = wins_df.groupby('Main Sport')['Odds'].mean()

print(average_winning_odds)

Main Sport
Baseball      1.575000
Basketball    1.961826
Cricket       1.330000
Football      1.787616
Ic            2.043333
Rugby         2.010000
Tabl          1.400000
Tennis        1.534286
Volleyball    1.730000
Name: Odds, dtype: float64


### optimal number of bets in a days

In [13]:
#optimal number of bets in a days for each sport type
bets = wins_df.groupby(['Date', 'Main Sport']).count().reset_index()

In [ ]:
# Calculate winnings for each row (this will depend on your specific logic)
# For example, if Status 'Won' means that the bet was correct,
# we could do something like this, where we multiply the bet 'Odd' by 1 or 0, depending on the 'Status':
df['Winnings'] = df.apply(lambda row: row['Odds'] if row['Status'] == 'win' else 0, axis=1)

# Now group by 'Date' and sum the 'Winnings'
daily_winnings = df.groupby('Date')['Winnings'].sum()

# Sort the results in descending order to get the days with most winnings at the top
sorted_daily_winnings = daily_winnings.sort_values(ascending=False, limit=10)

# The result is a Series with the index as the date and the values as the total winnings for that day
print(sorted_daily_winnings)

In [ ]:
sorted_daily_winnings.head(10)


In [21]:
# Filter the DataFrame for wins if sport is 'Volleyball'
df_volley = df[(df['Main Sport'] == 'Volleyball') & (df['Status'] == 'win')]

In [23]:
df_volley

,Date,Time,Bet Type,Main Sport,Sub Sport,Home,Away,Odds,Status,Winnings
90,2023-12-28,18:45,W1,Volleyball,Czech Republic. Extraliga,Ostrava,Aero Odolena Voda,1.58,win,1.58
91,2023-12-28,19:00,Total Under (200.5),Volleyball,Qatar Championship,Al Rayyan,Al Arabi Qatar,1.47,win,1.47
116,2023-12-27,13:00,Total Sets Under (3.5),Volleyball,South Korea Championship,Seoul Woori Card Wibee,KB Stars,2.20,win,2.20
277,2023-12-20,13:00,W1,Volleyball,South Korea Championship,Seoul Woori Card Wibee,Cheonan Hyundai Capital Skywalkers,1.56,win,1.56
372,2023-12-16,15:15,W2,Volleyball,Turkey Championship. Women,Galatasaray (Women),SigortaShop (Women) 4 Set,1.89,win,1.89
373,2023-12-16,15:15,W2,Volleyball,Turkey Championship. Women,Galatasaray (Women),SigortaShop (Women) 4 Set,1.54,win,1.54
374,2023-12-16,13:55,Total Over (188.5),Volleyball,Turkey Championship. Women,Galatasaray (Women),SigortaShop (Women),2.10,win,2.10
378,2023-12-16,12:15,Total Under (27),Volleyball,South Korea Championship. Women,Daejeon KGC (Women),Hyundai EC Hillstate (Women) 5 Set,2.26,win,2.26
380,2023-12-16,10:00,W2,Volleyball,South Korea Championship. Women,Daejeon KGC (Women),Hyundai EC Hillstate (Women),2.21,win,2.21
408,2023-12-15,12:55,W1,Volleyball,South Korea Championship,Daejeon Samsung Fire Bluefangs,Ansan OK 1 Set,1.27,win,1.27


In [24]:
df_volley.groupby('Bet Type')['Status'].count()

Bet Type
Total Over (188.5)        1
Total Sets Under (3.5)    1
Total Under (200.5)       1
Total Under (27)          1
W1                        7
W2                        3
Name: Status, dtype: int64


#optimal number of bets in a days
#types of bets with highest win ration per sport
#day with the highest number of losses
#optimum/average odd resulting onto a win 
#most profitable day
#time of day when most bets are placed
#time of day when bets placed are profitable/riskier

In [ ]:
# Filter out rows with invalid dates
df = df.dropna(subset=['Date'])

# Add a new column 'Day of Week'
df['Day of Week'] = df['date'].dt.dayofweek

# Filter the DataFrame for wins
wins_df = df[df['Status'] == 'win']

# Group by 'Day of Week' and count wins
wins_per_day = wins_df.groupby('Day of Week').size()

# Calculate average wins for each day (you might need total days per each day of the week for this)
total_days = df['Day of Week'].value_counts()

average_wins_per_day = wins_per_day / total_days

# Find the day with the highest average number of wins
day_with_most_average_wins = int(average_wins_per_day.idxmax())  # or average_wins_per_day.idxmax()

# Convert the day index to a readable string if necessary, e.g., using calendar.day_name


In [54]:
df.groupby('Day of Week').size()

Day of Week
0.0    99
1.0    40
2.0    66
3.0    47
4.0    24
5.0    70
6.0    47
dtype: int64

In [52]:
import calendar
day_name = calendar.day_name[day_with_most_average_wins]

print(f"The day with the highest average number of wins is: {day_name}")

The day with the highest average number of wins is: Wednesday


In [58]:
#find the sub sport if the main sport is Ic 
hockey_df = df[df['Main Sport'] == 'Ic']


In [ ]:
hockey_df.drop('Day', axis=1, inplace=True)

In [62]:
hockey_df.head()

,Date,Time,Bet Type,Main Sport,Sub Sport,Home,Away,Odds,Status,date,Day of Week
95,28.12.2023,17:10,W1,Ic,Hockey. Spengler Cup,Dynamo Pardubice,KalPa,2.42,win,NaT,NaN
260,21.12.2023,06:00,Total Over (5),Ic,Hockey. NHL,Los Angeles Kings,Seattle Kraken,1.92,loss,NaT,NaN
297,19.12.2023,17:00,W1,Ic,Hockey. Kontinental Hockey League,Avtomobilist Yekaterinburg,Traktor Chelyabinsk,3.21,loss,NaT,NaN
298,19.12.2023,17:00,Total Over (4),Ic,Hockey. Kontinental Hockey League,Avtomobilist Yekaterinburg,Traktor Chelyabinsk,1.00,100% refund,NaT,NaN
484,10.12.2023,23:00,W1,Ic,Hockey. AHL,Calgary Wranglers,Manitoba Moose,1.82,loss,2023-10-12,3.0


In [ ]:
soccer_df = df[df['Main Sport'] == 'Football']